# Personal Search 02

Busca de informações em livros usando FAISS

## Import packages

In [1]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_community.llms import Ollama
from IPython.display import display, Markdown
import logging

#logging.basicConfig(level=logging.INFO)

In [2]:
# limpar memória gpu
#import torch
#torch.cuda.empty_cache()

## Configuration

In [3]:
model = "llama3:8b"
#model = "mistral:7b"
embeddings_model = "nomic-embed-text:v1.5"
path_books = "./vector-store/books"
question = "Homem e seu egoismo"

## Retriever Instance

In [4]:
embeddings = OllamaEmbeddings(model=embeddings_model)

db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = False
retriever.search_kwargs["k"] = 2

## Search

In [5]:
sources = retriever.invoke(question)
livros = ''
for source in sources:
    source_name = source.metadata['source'].split('/')[-1]
    source_page = source.metadata['page']
    livros += f'**Fonte:** {source_name} - Página: {source_page}\n\n'
    livros += f'{source.page_content}\n\n'

In [6]:
# RAG
template = """
Cite trechos do contexto que tenha relação com "{question}". Não elabore, apenas cite.
Caso não encontre, apenas diga que não encontrou. 
Obedeça o formato "Citação do livro" - Fonte: Nome do Livro - Página: Número da Página

Contexto:
{context}

Responda em português.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = Ollama(model=model, temperature=0)
#llm = ChatOllama(model=model, temperature=0)

final_rag_chain = (
    {"context": itemgetter("livros"), 
     "question": itemgetter("question")} 
    | prompt
    | llm.bind(stop=["<|eot_id|>"])
    | StrOutputParser()
)
# llm.bind(stop=["<|eot_id|>"])

final_answer = final_rag_chain.invoke({"question":question, "livros":livros})

In [7]:
display(Markdown(f"**Tópico:** {question}\n\n"))
display(Markdown(final_answer))

**Tópico:** Homem e seu egoismo



Aqui estão as citações relacionadas ao contexto "Homem e seu egoismo":

**Fonte:** Conselhos para obreiros - Charles H Spurgeon.pdf - Página: 37
"O preguiçoso não tem direito ao púlpito. Ele é um instrumento de Satanás para condenar a alma dos homens."

**Fonte:** O Peregrino - John Bunyan.pdf - Página: (não especificada)
"Nenhum deles, porém, ousava fazê-lo." (referindo-se ao homem que deseja entrar no castelo)

In [8]:
sources = retriever.invoke(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source.metadata}"))
    #display(Markdown(f"**Trecho:** {source.page_content}"))

**Fonte:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 37}

**Fonte:** {'source': './books/import/O Peregrino - John Bunyan.pdf', 'page': 37}

In [9]:
sources = db.similarity_search_with_score(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source[0].page_content}"))
    display(Markdown(f"**Trecho:** {source[0].metadata}"))
    display(Markdown(f"**Score:** {source[1]}"))

**Fonte:** 8
O tipo de obreiro que Deus procura
Que tipo de homem o Mestre quer usar? Certamente é aquele que é um trabalhador.
O  grande  último dia  será  muito  difícil  para  aquele  que  não  trabalhar  arduamente  em  seu
ministério, pois terá de responder pelo seu ócio. Quem deseja uma vida fácil nunca deveria
pensar em ocupar o púlpito cristão. O púlpito não é o lugar dessa pessoa, e, quando ela ali
chegar, o único conselho que posso lhe dar é que saia dessa posição o mais rápido possível. Se,
por acaso, tal pessoa não deixar o púlpito, lembrá-la-ei das palavras de Jeú em relação a Jezabel:
“Jogai-a daí para baixo” (2Rs 9.33) — e penso que isso se aplica ao ministro preguiçoso. O
preguiçoso não tem direito ao púlpito. Ele é um instrumento de Satanás para condenar a alma dos
homens. Inteligente ministério exige esforço mental; o pregador deve usar a inteligência em seu
ensino, ler e estudar a fim de manter sua mente saudável. Não deve aborrecer as pessoas,
dizendo-lhes a verdade de um modo antiquado e inútil, sem que nada de novo e genuíno brote da
sua alma para fortalecer seu ensino. Acima de tudo, ele deve colocar seu coração na pregação.
Ele deve sentir o que prega; pregar a mensagem nunca deve lhe parecer uma tarefa fácil. Ele
deve se sentir, ao fim de uma mensagem, como se tivesse derramado a própria vida. Sua alma
deve estar na obra; todo o seu ser deve estar envolvido no esforço; toda a destreza que Deus lhe
deu deve ser usada vigorosamente na obra que foi confiada às suas mãos. Esse é o homem que
nós queremos. Levantar-se e pregar um sermão monótono como um ronco, para pessoas que
estão meio dormindo, meio acordadas, deve ser um trabalho muito triste. Eu me pergunto que
tipo de desculpa alguns darão, no final, por terem agido assim habitualmente. Proclamar um
credo estéril, explicar certas doutrinas usando argumentos lógicos, mas sem jamais incomodar a
consciência das pessoas, nunca condenar seus pecados nem adverti-las dos perigos que estão
correndo e nunca convidá-las a se achegarem ao Salvador com lágrimas e súplicas! Que obra
sem  poder!  O  que  será  de  tais  pregadores?  Deus  tenha  misericórdia  deles!  Queremos
trabalhadores, e não indolentes! Precisamos de homens que tenham o coração em chamas, e eu
lhes suplico que peçam a Deus que envie tais homens. A colheita nunca será ceifada por homens
que não trabalham; eles têm de arregaçar as mangas. Isso significa que eles devem deixar de lado

**Trecho:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 37}

**Score:** 373.4029235839844

**Fonte:** mas você pode ver que o fogo, assim mesmo, fica sempre maior e mais quente.
Você também verá a razão disso.
Então ele o levou para trás da parede, onde se via um homem com um vaso
de óleo nas mãos. Ele, em segredo, derramava continuamente óleo no fogo.
Cristão perguntou:
— O que isso significa?
— Este é Cristo, que continuamente, com o óleo da sua graça, mantém viva a
obra já iniciada no coração, e por meio dessa obra, a despeito de tudo o que pode
fazer o Diabo, as almas dos crentes se provam piedosas. Mas você observou
também que o homem fica atrás da parede para alimentar o fogo. Isso é para
ensinar-lhe que é difícil para aquele que é tentado ver como essa obra graciosa
se conserva na alma.
Vi também que Intérprete tomava-o novamente pela mão, levando-o até um
lugar agradável, onde se erguia um imponente castelo, belo de admirar. Cristão
exultou diante da visão. Viu ele também que, no terraço do castelo, algumas
pessoas caminhavam, vestidas todas de ouro.
— Podemos entrar? — perguntou Cristão.
Intérprete então lhe tomou a mão e o levou até a porta do castelo, e eis que, à
porta, havia um grande ajuntamento de homens, todos desejosos de entrar.
Nenhum deles, porém, ousava fazê-lo. Ali também estava sentado um homem, a
pouca distância da porta, ao lado de uma mesinha, e sobre esta se via um tinteiro
e um livro, no qual ele anotava o nome daquele que haveria de entrar.
Cristão viu também que no vão da porta havia muitos homens que, trajando
armaduras, barravam a passagem, determinados a fazer todo dano e mal que
pudessem àquele que tentasse entrar. Cristão ficou ali absorto. Por fim, quando
todos começaram a recuar com medo dos homens armados, Cristão viu um
homem de semblante bem resoluto aproximar-se do que estava sentado
escrevendo e disse:
— Anote o meu nome, senhor.
Viu então que o homem desembainhava a espada e colocava um capacete na
cabeça, avançando rumo à porta, contra os homens armados, que sobre ele
caíram com força mortal. O homem, porém, não se deixou esmorecer e,
mesmo caído, golpeava e talhava ferozmente. Assim, depois de ser ferido e ferir
muitos dos homens que tentavam evitar a sua entrada, conseguiu abrir caminho
em meio a todos eles, entrando no palácio.
Ouviu-se, então, uma bela voz vinda dos que estavam lá dentro, dos três
mesmos que caminhavam no terraço do palácio, e dizia:
 
Entre, entre;
Glória aqui você terá para sempre.

**Trecho:** {'source': './books/import/O Peregrino - John Bunyan.pdf', 'page': 37}

...

**Trecho:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 64}

**Score:** 389.10858154296875

**Fonte:** 1Alexander Pope, Ensaio sobre o homem, v. 3, epístola IV.

**Trecho:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 29}

**Score:** 395.8448486328125